In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
test_df= pd.read_csv("test2.csv") #reading a txt file with single datapoint

# Predict_outcome_function:

In [ ]:
def final1(df):
  import time
  start= time.time()
  import pandas as pd
  import numpy as np
    
  #if 'pandas.core.series.Series' is given instead of dataframe:
  if isinstance(df, pd.core.series.Series):
    values= list((df.to_frame()).iloc[:, 0]) #get_values
    cols= list(df.to_frame().index) #get index
    df= pd.DataFrame([values], columns = cols)
  else:
    pass

  test_id= (df.iloc[0]['ID']).astype(int)
  test_df= df.drop(['ID'], axis=1)

  #data cleaning:
  # load pickle with all removable_features
  import pickle
  with open('pickle/const_feat.pkl', 'rb') as o:
      const_feat = pickle.load(o)
  with open('pickle/quasi_const_feat.pkl', 'rb') as o:
      quasi_const_feat = pickle.load(o)
  with open('pickle/correlated_columns.pkl', 'rb') as o:
      correlated_columns = pickle.load(o)
  with open('pickle/sparse_columns.pkl', 'rb') as o:
      sparse_columns = pickle.load(o)

  remove_cols= const_feat+ quasi_const_feat+ correlated_columns+ sparse_columns

  def drop_fetures(df):
    df.drop(set(remove_cols), axis=1, inplace= True)

  drop_fetures(test_df)

  #adding new features:
  def count_of_zeros(df):
    df.insert((df.shape[1]), 'count_zeros', (df == 0).astype(int).sum(axis=1))

  def count_of_non_zeros(df):
    df.insert((df.shape[1]), 'count_non_zeros', (df != 0).astype(int).sum(axis=1))

  def age_below_23(df):
    below_23= []
    for i in (df['var15']):
      if i < 23:
        below_23.append(1)
      else:
        below_23.append(0)

    df['below_23']= below_23

  def age_above_80(df):
    above_80= []
    for i in (df['var15']):
      if i > 80:
        above_80.append(1)
      else:
        above_80.append(0)

    df['above_80']= above_80

  def saldo_var30_0_3(df):
    saldo_var30_0_3= []
    for i in (df['saldo_var30']):
      if ((i == 0)or(i ==3)):
        saldo_var30_0_3.append(1)
      else:
        saldo_var30_0_3.append(0)

    df['saldo_var30_0_3']= saldo_var30_0_3

  def value_var38(df):
    value_var38= []
    for i in (df['var38']):
      if (i == 117310.979016494):
        value_var38.append(1)
      else:
        value_var38.append(0)

    df['value_var38']= value_var38

  def value_saldo_medio_var5_ult3(df):
    value_saldo_medio_var5_ult3= []
    for i in (df['saldo_medio_var5_ult3']):
      if (i == 0.0):
        value_saldo_medio_var5_ult3.append(1)
      else:
        value_saldo_medio_var5_ult3.append(0)

    df['value_saldo_medio_var5_ult3']= value_saldo_medio_var5_ult3

  count_of_zeros(test_df)

  count_of_non_zeros(test_df)

  age_below_23(test_df)

  age_above_80(test_df)

  saldo_var30_0_3(test_df)

  value_var38(test_df)

  value_saldo_medio_var5_ult3(test_df)

  #load models:
  import tensorflow as tf

  with open('pickle/clf_rf.pkl', 'rb') as o: #random_forest
      clf_rf = pickle.load(o)
  with open('pickle/clf_xgbrf.pkl', 'rb') as o: #xgbrfclassifier
      clf_xgbrf = pickle.load(o)
  with open('pickle/clf_xgb.pkl', 'rb') as o:#xgboost
      clf_xgb = pickle.load(o)
  with open('pickle/clf_lgbm.pkl', 'rb') as o:#lgbmclassifier
      clf_lgbm = pickle.load(o)
  with open('pickle/clf_adb.pkl', 'rb') as o:#adbclassifier
      clf_adb = pickle.load(o)
  model = tf.keras.models.load_model('pickle/dl_model')#neuaral network model

  with open('pickle/scaler.pkl', 'rb') as o:
      scaler = pickle.load(o)

  #preparing data for deep learning:
  print(test_df)
  test_df_dl= scaler.transform(test_df)
  X_test_dl = test_df_dl.reshape(test_df.shape[0], test_df.shape[1], 1)

  # get prediction_probablity:
  y_test_pred_rf = clf_rf.predict_proba(test_df)[:,1]
  y_test_pred_xgbrf = clf_xgbrf.predict_proba(test_df)[:,1]
  y_test_pred_xgb = clf_xgb.predict_proba(test_df)[:,1]
  y_test_pred_lgbm = clf_lgbm.predict_proba(test_df)[:,1]
  y_test_pred_adb = clf_adb.predict_proba(test_df) [:,1]

  y_test_pred= model.predict(X_test_dl) #dl model
  y_test_pred_dl= []
  for i in range(y_test_pred.shape[0]):
    y_test_pred_dl.append(y_test_pred[i][0])

  y_test_pred= (y_test_pred_rf+ y_test_pred_xgbrf+ y_test_pred_xgb+ y_test_pred_lgbm+ y_test_pred_adb+ y_test_pred_dl )/6

  #make prediction
  prediction=[]
  for i in y_test_pred:
    if i > 0.5:
      prediction.append("UNSATISFIED")
    else:
      prediction.append("SATISFIED")
      
  if len(prediction)==1:
    prediction= prediction[0]
  else:
    pass
  prediction="prediction: {}".format(prediction)

  end= time.time()
  execution_time= end-start
  execution_time= "time of execution :{} seconds".format(end-start)
  return prediction, execution_time

In [ ]:
result= final1(test_df)

In [ ]:
result

('prediction: SATISFIED', 'time of execution :0.5045850276947021 seconds')

# performance_metric_function:

In [ ]:
train_df= pd.read_csv("train.csv")
y_true= train_df['TARGET']
train_df=train_df.drop('TARGET', axis=1)

In [ ]:
def final2(df,y_true):
  import pandas as pd
  import numpy as np
  from sklearn.metrics import roc_curve, auc
  
  y_true=y_true

  #if 'pandas.core.series.Series' is given instead of dataframe:
  if isinstance(df, pd.core.series.Series):
    values= list((df.to_frame()).iloc[:, 0]) #get_values
    cols= list(df.to_frame().index) #get index
    df= pd.DataFrame([values], columns = cols)
  else:
    pass

  if 'TARGET' in df.columns:
    y_true= df['TARGET']
    df=df.drop(['TARGET'], axis=1)
  else:
    pass

  if 'ID' in df.columns:
    test_id= df['ID'] 
    df= df.drop(['ID'], axis=1)
  else:
    pass

  #data cleaning:
  # load pickle with all removable_features
  import pickle
  with open('pickle/const_feat.pkl', 'rb') as o:
      const_feat = pickle.load(o)
  with open('pickle/quasi_const_feat.pkl', 'rb') as o:
      quasi_const_feat = pickle.load(o)
  with open('pickle/correlated_columns.pkl', 'rb') as o:
      correlated_columns = pickle.load(o)
  with open('pickle/sparse_columns.pkl', 'rb') as o:
      sparse_columns = pickle.load(o)

  remove_cols= const_feat+ quasi_const_feat+ correlated_columns+ sparse_columns

  def drop_features(df):
    df.drop(set(remove_cols), axis=1, inplace= True)

  drop_features(df)

  #adding new features:
  def count_of_zeros(df):
    df.insert((df.shape[1]), 'count_zeros', (df == 0).astype(int).sum(axis=1))

  def count_of_non_zeros(df):
    df.insert((df.shape[1]), 'count_non_zeros', (df != 0).astype(int).sum(axis=1))

  def age_below_23(df):
    below_23= []
    for i in (df['var15']):
      if i < 23:
        below_23.append(1)
      else:
        below_23.append(0)

    df['below_23']= below_23

  def age_above_80(df):
    above_80= []
    for i in (df['var15']):
      if i > 80:
        above_80.append(1)
      else:
        above_80.append(0)

    df['above_80']= above_80

  def saldo_var30_0_3(df):
    saldo_var30_0_3= []
    for i in (df['saldo_var30']):
      if ((i == 0)or(i ==3)):
        saldo_var30_0_3.append(1)
      else:
        saldo_var30_0_3.append(0)

    df['saldo_var30_0_3']= saldo_var30_0_3

  def value_var38(df):
    value_var38= []
    for i in (df['var38']):
      if (i == 117310.979016494):
        value_var38.append(1)
      else:
        value_var38.append(0)

    df['value_var38']= value_var38

  def value_saldo_medio_var5_ult3(df):
    value_saldo_medio_var5_ult3= []
    for i in (df['saldo_medio_var5_ult3']):
      if (i == 0.0):
        value_saldo_medio_var5_ult3.append(1)
      else:
        value_saldo_medio_var5_ult3.append(0)

    df['value_saldo_medio_var5_ult3']= value_saldo_medio_var5_ult3

  count_of_zeros(df)

  count_of_non_zeros(df)

  age_below_23(df)

  age_above_80(df)

  saldo_var30_0_3(df)

  value_var38(df)

  value_saldo_medio_var5_ult3(df)

  #load models:
  import tensorflow as tf

  with open('pickle/clf_rf.pkl', 'rb') as o: #random_forest
      clf_rf = pickle.load(o)
  with open('pickle/clf_xgbrf.pkl', 'rb') as o: #xgbrfclassifier
      clf_xgbrf = pickle.load(o)
  with open('pickle/clf_xgb.pkl', 'rb') as o:#xgboost
      clf_xgb = pickle.load(o)
  with open('pickle/clf_lgbm.pkl', 'rb') as o:#lgbmclassifier
      clf_lgbm = pickle.load(o)
  with open('pickle/clf_adb.pkl', 'rb') as o:#adbclassifier
      clf_adb = pickle.load(o)
  model = tf.keras.models.load_model('pickle/dl_model')#neuaral network model

  with open('pickle/scaler.pkl', 'rb') as o:
      scaler = pickle.load(o)

  #preparing data for deep learning:
  df_dl= scaler.transform(df)
  X_dl = df_dl.reshape(df.shape[0], df.shape[1], 1)

  # get prediction_probablity:
  y_test_pred_rf = clf_rf.predict_proba(df)[:,1]
  y_test_pred_xgbrf = clf_xgbrf.predict_proba(df)[:,1]
  y_test_pred_xgb = clf_xgb.predict_proba(df)[:,1]
  y_test_pred_lgbm = clf_lgbm.predict_proba(df)[:,1]
  y_test_pred_adb = clf_adb.predict_proba(df) [:,1]

  y_test_pred= model.predict(X_dl) #dl model
  y_test_pred_dl= []
  for i in range(y_test_pred.shape[0]):
    y_test_pred_dl.append(y_test_pred[i][0])

  y_true= y_true
  y_test_pred= (y_test_pred_rf+ y_test_pred_xgbrf+ y_test_pred_xgb+ y_test_pred_lgbm+ y_test_pred_adb+ y_test_pred_dl )/6

  test_fpr, test_tpr, tr_thresholds = roc_curve(y_true, y_test_pred)

  score= auc(test_fpr, test_tpr)
  return score

In [ ]:
final2(train_df,y_true)

0.915757344901625